In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from MyEarlyStopping import MyEarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

## Training dataset (40%)

In [3]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [4]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)

In [5]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
subject = 2
train_image = 10 #10:20%, 20: 40%, 30:60%

In [7]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10.], dtype=float32)

## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)


In [10]:
ALL_Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
ALL_Val_Inputs = np.array(ALL_Val_Inputs)

In [11]:
select_indexs_val = participant_class==subject
Val_Inputs = ALL_Val_Inputs[select_indexs_val]
Val_command_class = Val_command_class[select_indexs_val]
sum(Val_command_class)

array([11., 11., 10., 11., 11., 11., 11., 10., 11., 11.], dtype=float32)

## Test dataset (20%)

In [12]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [13]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)

In [14]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [15]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

In [16]:
# sum(Test_command_class_2)

## Speaker 6 Test dataset 

In [17]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [18]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [19]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [20]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [21]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [22]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [23]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [24]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [25]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [26]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Command_Acc_p15','Command_Acc_p1','Command_Acc_p2',
#                                      'Command_Acc_p3','Command_Acc_p4','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_single_model.csv')
# Perfomance
Perfomance                                                        

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
5,Subjet_1,14.131006,0.5672,0.9279,0.2870,0.4696,0.5780,0.57,0.25,0.328,0.2673,10
6,Subjet_1,13.651466,0.5930,0.9459,0.2685,0.5130,0.6239,0.61,0.43,0.440,0.2673,10
7,Subjet_1,12.100064,0.5727,0.9279,0.2870,0.4957,0.5413,0.61,0.41,0.384,0.2574,10
8,Subjet_1,13.176442,0.6151,0.9459,0.3611,0.4957,0.6147,0.66,0.41,0.408,0.2871,10
9,Subjet_1,13.929373,0.6446,0.9369,0.4722,0.5391,0.6147,0.66,0.48,0.408,0.2673,10


# Create model 

In [27]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Single model （size:10）

In [28]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_2",'Size':'10','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_2_model_size10_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_2_size10_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_2_size10_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
2/2 [==============================] - 2s 940ms/step - loss: 2.4774 - accuracy: 0.0800 - val_loss: 2.2043 - val_accuracy: 0.4907
Epoch 2/10
2/2 [==============================] - 1s 275ms/step - loss: 2.1630 - accuracy: 0.6000 - val_loss: 2.0980 - val_accuracy: 0.5926
Epoch 3/10
2/2 [==============================] - 1s 285ms/step - loss: 2.0399 - accuracy: 0.7100 - val_loss: 2.0006 - val_accuracy: 0.7500
Epoch 4/10
2/2 [==============================] - 1s 277ms/step - loss: 1.9459 - accuracy: 0.8200 - val_loss: 1.9270 - val_accuracy: 0.7870
Epoch 5/10
2/2 [==============================] - 1s 263ms/step - loss: 1.8744 - accuracy: 0.8500 - val_loss: 1.8859 - val_accuracy: 0.7963
Epoch 6/10
2/2 [==============================] - 0s 92ms/step - loss: 1.8247 - accuracy: 0.8300 - val_loss: 1.8430 - val_accuracy: 0.7963
Epoch 7/10
2/2 [==============================] - 1s 293ms/step - loss: 1.7799 - accuracy: 0.8300 - val_loss: 1.7938 - val_accuracy: 0.8796
Epoch 8/10
2/2 [=====

2/2 [==============================] - 1s 272ms/step - loss: 2.0835 - accuracy: 0.6900 - val_loss: 2.0269 - val_accuracy: 0.6852
Epoch 4/10
2/2 [==============================] - 1s 297ms/step - loss: 1.9883 - accuracy: 0.6900 - val_loss: 1.9575 - val_accuracy: 0.7037
Epoch 5/10
2/2 [==============================] - 0s 101ms/step - loss: 1.9166 - accuracy: 0.6900 - val_loss: 1.9022 - val_accuracy: 0.7037
Epoch 6/10
2/2 [==============================] - 1s 414ms/step - loss: 1.8579 - accuracy: 0.7100 - val_loss: 1.8703 - val_accuracy: 0.7407
Epoch 7/10
2/2 [==============================] - 1s 284ms/step - loss: 1.8175 - accuracy: 0.8100 - val_loss: 1.8379 - val_accuracy: 0.8241
Epoch 8/10
2/2 [==============================] - 1s 288ms/step - loss: 1.7776 - accuracy: 0.9000 - val_loss: 1.8053 - val_accuracy: 0.8611
Epoch 9/10
2/2 [==============================] - 0s 98ms/step - loss: 1.7450 - accuracy: 0.9600 - val_loss: 1.7832 - val_accuracy: 0.8611
Epoch 10/10
2/2 [===============

2/2 [==============================] - 1s 656ms/step - loss: 2.4527 - accuracy: 0.1000 - val_loss: 2.2609 - val_accuracy: 0.1389
Epoch 2/10
2/2 [==============================] - 1s 284ms/step - loss: 2.2026 - accuracy: 0.2100 - val_loss: 2.1219 - val_accuracy: 0.5278
Epoch 3/10
2/2 [==============================] - 1s 277ms/step - loss: 2.0653 - accuracy: 0.5800 - val_loss: 2.0014 - val_accuracy: 0.5741
Epoch 4/10
2/2 [==============================] - 1s 279ms/step - loss: 1.9540 - accuracy: 0.6200 - val_loss: 1.9357 - val_accuracy: 0.6111
Epoch 5/10
2/2 [==============================] - 1s 379ms/step - loss: 1.8750 - accuracy: 0.6700 - val_loss: 1.8645 - val_accuracy: 0.7130
Epoch 6/10
2/2 [==============================] - 1s 280ms/step - loss: 1.8092 - accuracy: 0.8300 - val_loss: 1.8173 - val_accuracy: 0.8611
Epoch 7/10
2/2 [==============================] - 0s 92ms/step - loss: 1.7624 - accuracy: 0.8800 - val_loss: 1.7761 - val_accuracy: 0.8426
Epoch 8/10
2/2 [================

Epoch 4/10
2/2 [==============================] - 1s 281ms/step - loss: 1.9728 - accuracy: 0.7300 - val_loss: 1.9501 - val_accuracy: 0.7315
Epoch 5/10
2/2 [==============================] - 1s 303ms/step - loss: 1.9150 - accuracy: 0.7600 - val_loss: 1.8959 - val_accuracy: 0.7870
Epoch 6/10
2/2 [==============================] - 1s 280ms/step - loss: 1.8523 - accuracy: 0.7900 - val_loss: 1.8441 - val_accuracy: 0.8333
Epoch 7/10
2/2 [==============================] - 0s 91ms/step - loss: 1.7980 - accuracy: 0.8500 - val_loss: 1.8187 - val_accuracy: 0.8148
Epoch 8/10
2/2 [==============================] - 0s 89ms/step - loss: 1.7753 - accuracy: 0.8600 - val_loss: 1.7828 - val_accuracy: 0.7963
Epoch 9/10
2/2 [==============================] - 1s 274ms/step - loss: 1.7357 - accuracy: 0.8500 - val_loss: 1.7537 - val_accuracy: 0.8611
Epoch 10/10
2/2 [==============================] - 0s 88ms/step - loss: 1.7022 - accuracy: 0.9000 - val_loss: 1.7343 - val_accuracy: 0.8611
Epoch 1/10
2/2 [======

In [29]:
resnet_model = tf.keras.models.load_model('Initial_subject_2_model_size10_0608.h5')
predictions = resnet_model.predict(Test_Inputs_2)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_2, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_2, axis=1)),4)
print(acc_c)

0.8611


In [30]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
5,Subjet_1,14.131006,0.5672,0.9279,0.2870,0.4696,0.5780,0.57,0.25,0.328,0.2673,10
6,Subjet_1,13.651466,0.5930,0.9459,0.2685,0.5130,0.6239,0.61,0.43,0.440,0.2673,10
7,Subjet_1,12.100064,0.5727,0.9279,0.2870,0.4957,0.5413,0.61,0.41,0.384,0.2574,10
8,Subjet_1,13.176442,0.6151,0.9459,0.3611,0.4957,0.6147,0.66,0.41,0.408,0.2871,10
9,Subjet_1,13.929373,0.6446,0.9369,0.4722,0.5391,0.6147,0.66,0.48,0.408,0.2673,10


In [31]:
best_index

8

## Single model （size:20）

In [32]:
subject = 2
train_image = 20 #10:20%, 20: 40%, 30:60%

In [33]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([20., 20., 20., 20., 20., 20., 20., 20., 20., 20.], dtype=float32)

In [34]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_2",'Size':'20','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_2_model_size20_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_2_size20_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_2_size20_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
4/4 [==============================] - 2s 379ms/step - loss: 2.4831 - accuracy: 0.0800 - val_loss: 2.2018 - val_accuracy: 0.3796
Epoch 2/10
4/4 [==============================] - 1s 182ms/step - loss: 2.1669 - accuracy: 0.3700 - val_loss: 2.0540 - val_accuracy: 0.5741
Epoch 3/10
4/4 [==============================] - 1s 169ms/step - loss: 2.0325 - accuracy: 0.5900 - val_loss: 1.9833 - val_accuracy: 0.6667
Epoch 4/10
4/4 [==============================] - 1s 172ms/step - loss: 1.9646 - accuracy: 0.6300 - val_loss: 1.9286 - val_accuracy: 0.6944
Epoch 5/10
4/4 [==============================] - 1s 169ms/step - loss: 1.9172 - accuracy: 0.7200 - val_loss: 1.8805 - val_accuracy: 0.8148
Epoch 6/10
4/4 [==============================] - 0s 76ms/step - loss: 1.8717 - accuracy: 0.8000 - val_loss: 1.8546 - val_accuracy: 0.7685
Epoch 7/10
4/4 [==============================] - 1s 174ms/step - loss: 1.8409 - accuracy: 0.8050 - val_loss: 1.8243 - val_accuracy: 0.8333
Epoch 8/10
4/4 [=====

Epoch 3/10
4/4 [==============================] - 1s 177ms/step - loss: 2.1192 - accuracy: 0.5850 - val_loss: 2.0756 - val_accuracy: 0.6944
Epoch 4/10
4/4 [==============================] - 1s 189ms/step - loss: 2.0657 - accuracy: 0.6550 - val_loss: 2.0356 - val_accuracy: 0.7685
Epoch 5/10
4/4 [==============================] - 1s 169ms/step - loss: 2.0225 - accuracy: 0.7700 - val_loss: 2.0034 - val_accuracy: 0.8889
Epoch 6/10
4/4 [==============================] - 0s 78ms/step - loss: 1.9956 - accuracy: 0.8150 - val_loss: 1.9878 - val_accuracy: 0.6204
Epoch 7/10
4/4 [==============================] - 0s 78ms/step - loss: 1.9740 - accuracy: 0.6150 - val_loss: 1.9657 - val_accuracy: 0.7315
Epoch 8/10
4/4 [==============================] - 0s 76ms/step - loss: 1.9494 - accuracy: 0.8000 - val_loss: 1.9393 - val_accuracy: 0.8796
Epoch 9/10
4/4 [==============================] - 0s 78ms/step - loss: 1.9296 - accuracy: 0.8550 - val_loss: 1.9186 - val_accuracy: 0.8889
Epoch 10/10
4/4 [=======

4/4 [==============================] - 0s 84ms/step - loss: 1.6756 - accuracy: 0.8100 - val_loss: 1.6498 - val_accuracy: 0.7778
Epoch 6/10
4/4 [==============================] - 0s 86ms/step - loss: 1.6155 - accuracy: 0.7800 - val_loss: 1.6045 - val_accuracy: 0.8241
Epoch 7/10
4/4 [==============================] - 0s 79ms/step - loss: 1.5619 - accuracy: 0.8500 - val_loss: 1.5502 - val_accuracy: 0.8704
Epoch 8/10
4/4 [==============================] - 1s 176ms/step - loss: 1.5084 - accuracy: 0.8550 - val_loss: 1.4935 - val_accuracy: 0.8981
Epoch 9/10
4/4 [==============================] - 1s 181ms/step - loss: 1.4556 - accuracy: 0.8900 - val_loss: 1.4486 - val_accuracy: 0.9352
Epoch 10/10
4/4 [==============================] - 0s 79ms/step - loss: 1.4081 - accuracy: 0.9200 - val_loss: 1.4038 - val_accuracy: 0.9259
Epoch 1/10
4/4 [==============================] - 1s 178ms/step - loss: 1.3575 - accuracy: 0.9550 - val_loss: 1.3536 - val_accuracy: 0.9537
Epoch 2/10
4/4 [==================

Epoch 4/10
4/4 [==============================] - 0s 79ms/step - loss: 2.0504 - accuracy: 0.4900 - val_loss: 2.0114 - val_accuracy: 0.4352
Epoch 5/10
4/4 [==============================] - 1s 179ms/step - loss: 1.9938 - accuracy: 0.5050 - val_loss: 1.9607 - val_accuracy: 0.6667
Epoch 6/10
4/4 [==============================] - 1s 172ms/step - loss: 1.9467 - accuracy: 0.7600 - val_loss: 1.9271 - val_accuracy: 0.7222
Epoch 7/10
4/4 [==============================] - 0s 82ms/step - loss: 1.9106 - accuracy: 0.6950 - val_loss: 1.8926 - val_accuracy: 0.7222
Epoch 8/10
4/4 [==============================] - 1s 171ms/step - loss: 1.8763 - accuracy: 0.7700 - val_loss: 1.8680 - val_accuracy: 0.7315
Epoch 9/10
4/4 [==============================] - 1s 177ms/step - loss: 1.8543 - accuracy: 0.7450 - val_loss: 1.8447 - val_accuracy: 0.7593
Epoch 10/10
4/4 [==============================] - 1s 180ms/step - loss: 1.8352 - accuracy: 0.7850 - val_loss: 1.8277 - val_accuracy: 0.8426
Epoch 1/10
4/4 [=====

In [35]:
resnet_model = tf.keras.models.load_model('Initial_subject_2_model_size20_0608.h5')
predictions = resnet_model.predict(Test_Inputs_2)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_2, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_2, axis=1)),4)
print(acc_c)

0.9074


In [36]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
5,Subjet_1,14.131006,0.5672,0.9279,0.2870,0.4696,0.5780,0.57,0.25,0.328,0.2673,10
6,Subjet_1,13.651466,0.5930,0.9459,0.2685,0.5130,0.6239,0.61,0.43,0.440,0.2673,10
7,Subjet_1,12.100064,0.5727,0.9279,0.2870,0.4957,0.5413,0.61,0.41,0.384,0.2574,10
8,Subjet_1,13.176442,0.6151,0.9459,0.3611,0.4957,0.6147,0.66,0.41,0.408,0.2871,10
9,Subjet_1,13.929373,0.6446,0.9369,0.4722,0.5391,0.6147,0.66,0.48,0.408,0.2673,10


In [37]:
best_index

4

## Single model （size:30）

In [38]:
subject = 2
train_image = 30 #10:20%, 20: 40%, 30:60%

In [39]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([30., 30., 30., 30., 30., 30., 30., 30., 30., 30.], dtype=float32)

In [40]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_2",'Size':'30','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_2_model_size30_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_2_size30_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_2_size30_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
5/5 [==============================] - 2s 345ms/step - loss: 2.2726 - accuracy: 0.2400 - val_loss: 2.0401 - val_accuracy: 0.3889
Epoch 2/10
5/5 [==============================] - 1s 157ms/step - loss: 1.9797 - accuracy: 0.6167 - val_loss: 1.8767 - val_accuracy: 0.7500
Epoch 3/10
5/5 [==============================] - 1s 162ms/step - loss: 1.8455 - accuracy: 0.7333 - val_loss: 1.7848 - val_accuracy: 0.7685
Epoch 4/10
5/5 [==============================] - 1s 177ms/step - loss: 1.7533 - accuracy: 0.8433 - val_loss: 1.7087 - val_accuracy: 0.8889
Epoch 5/10
5/5 [==============================] - 1s 162ms/step - loss: 1.6802 - accuracy: 0.9167 - val_loss: 1.6480 - val_accuracy: 0.9259
Epoch 6/10
5/5 [==============================] - 1s 168ms/step - loss: 1.6200 - accuracy: 0.9400 - val_loss: 1.5908 - val_accuracy: 0.9352
Epoch 7/10
5/5 [==============================] - 1s 167ms/step - loss: 1.5597 - accuracy: 0.9400 - val_loss: 1.5297 - val_accuracy: 0.9722
Epoch 8/10
5/5 [====

5/5 [==============================] - 0s 97ms/step - loss: 1.5418 - accuracy: 0.9700 - val_loss: 1.5163 - val_accuracy: 0.9722
Epoch 9/10
5/5 [==============================] - 1s 170ms/step - loss: 1.4890 - accuracy: 0.9700 - val_loss: 1.4616 - val_accuracy: 0.9815
Epoch 10/10
5/5 [==============================] - 1s 181ms/step - loss: 1.4329 - accuracy: 0.9833 - val_loss: 1.4066 - val_accuracy: 0.9907
Epoch 1/10
5/5 [==============================] - 1s 161ms/step - loss: 1.3757 - accuracy: 0.9933 - val_loss: 1.3572 - val_accuracy: 0.9907
Epoch 2/10
5/5 [==============================] - 0s 82ms/step - loss: 1.3235 - accuracy: 0.9867 - val_loss: 1.3052 - val_accuracy: 0.9907
Epoch 3/10
5/5 [==============================] - 0s 83ms/step - loss: 1.2665 - accuracy: 0.9900 - val_loss: 1.2513 - val_accuracy: 0.9907
Epoch 4/10
5/5 [==============================] - 0s 86ms/step - loss: 1.2146 - accuracy: 0.9900 - val_loss: 1.2037 - val_accuracy: 0.9907
Epoch 5/10
5/5 [==================

Epoch 7/10
5/5 [==============================] - 0s 83ms/step - loss: 1.0791 - accuracy: 0.9967 - val_loss: 1.0870 - val_accuracy: 0.9907
Epoch 8/10
5/5 [==============================] - 0s 84ms/step - loss: 1.0362 - accuracy: 0.9967 - val_loss: 1.0422 - val_accuracy: 0.9907
Epoch 9/10
5/5 [==============================] - 0s 82ms/step - loss: 0.9919 - accuracy: 1.0000 - val_loss: 1.0010 - val_accuracy: 0.9907
Epoch 10/10
5/5 [==============================] - 1s 145ms/step - loss: 0.9502 - accuracy: 1.0000 - val_loss: 0.9585 - val_accuracy: 0.9907
Epoch 1/10
5/5 [==============================] - 2s 348ms/step - loss: 2.3417 - accuracy: 0.2767 - val_loss: 2.1211 - val_accuracy: 0.5370
Epoch 2/10
5/5 [==============================] - 1s 159ms/step - loss: 2.0667 - accuracy: 0.5600 - val_loss: 1.9927 - val_accuracy: 0.6204
Epoch 3/10
5/5 [==============================] - 1s 174ms/step - loss: 1.9540 - accuracy: 0.7333 - val_loss: 1.9027 - val_accuracy: 0.8056
Epoch 4/10
5/5 [======

In [41]:
resnet_model = tf.keras.models.load_model('Initial_subject_2_model_size30_0608.h5')
predictions = resnet_model.predict(Test_Inputs_2)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_2, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_2, axis=1)),4)
print(acc_c)

0.9444


In [42]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
5,Subjet_1,14.131006,0.5672,0.9279,0.2870,0.4696,0.5780,0.57,0.25,0.328,0.2673,10
6,Subjet_1,13.651466,0.5930,0.9459,0.2685,0.5130,0.6239,0.61,0.43,0.440,0.2673,10
7,Subjet_1,12.100064,0.5727,0.9279,0.2870,0.4957,0.5413,0.61,0.41,0.384,0.2574,10
8,Subjet_1,13.176442,0.6151,0.9459,0.3611,0.4957,0.6147,0.66,0.41,0.408,0.2871,10
9,Subjet_1,13.929373,0.6446,0.9369,0.4722,0.5391,0.6147,0.66,0.48,0.408,0.2673,10


In [43]:
best_index

9

In [44]:
Perfomance.to_csv('Performance_0608_single_model.csv',index=False)